---
# Chapter 6
## Design Patterns with First-Class Functions

En este capítulo se ilustra como implementar patrones de diseño utilizando las _"First-Class Functions"_ en Python, tales como _Refactoring Strategy_ y _Command Strategy_

---

## Case Study: _Refactoring Strategy_

- Classic Strategy
- Function Oriented Strategy

---

## Classic Strategy

---
### Example 6-1. Implemetation Order class with pluggable discount strategies

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.quantity * self.price


class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum([item.total() for item in self.cart])
        return self._total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total:{:2f} due:{:2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        raise NotImplementedError('Method discount not implemented')


class FidelityPromo(Promotion):

    def discount(self, order):
        if order.customer.fidelity >= 1000:
            return order.total() * 0.05
        return 0
        #return order.total() * 0.5 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1

        return discount


class LargeOrderPromo(Promotion):

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07

        return 0

---
### Example 6-2. Sample usage of Order class with different promotions class applied

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [3]:
Order(joe, cart, FidelityPromo())

<Order total:42.000000 due:42.000000>

In [4]:
Order(ann, cart, FidelityPromo())

<Order total:42.000000 due:39.900000>

In [5]:
banana_cart = [
    LineItem('banana', 30, .5),
    LineItem('apple', 10, 1.5)
]

In [6]:
Order(joe, banana_cart, BulkItemPromo())

<Order total:30.000000 due:28.500000>

In [7]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [8]:
Order(joe, long_order, LargeOrderPromo())

<Order total:10.000000 due:9.300000>

In [9]:
Order(joe, cart, LargeOrderPromo())

<Order total:42.000000 due:42.000000>

## Function Oriented Strategy

---
### Example 6-3. Order class with discount strategies implemented as fuctions

In [10]:
Customer = namedtuple('Customer', 'name fidelity')

class ListItem:
    def __init__(self, product, quantity, price):
        self.producto = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.quantity * price

    
class Order:  # The context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum([item.total() for item in self.cart])
        return self._total

    def due(self):
        if not self.promotion:
            discount = 0
        else:
            discount = self.promotion(self)

        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:2f} due: {:2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order: Order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order: Order):
    discount = 0
    for item in order.cart:
        if item.quantity >=20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order: Order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

---
### Example 6-4. Sample usage of Order class with promotions as fuctions

In [11]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [12]:
Order(joe, cart, fidelity_promo)

<Order total: 42.000000 due: 42.000000>

In [13]:
Order(ann, cart, fidelity_promo)

<Order total: 42.000000 due: 39.900000>

In [14]:
banana_cart = [
    LineItem('banana', 30, .5),
    LineItem('apple', 10, 1.5)
]

In [15]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.000000 due: 28.500000>

In [16]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [17]:
Order(ann, long_order, large_order_promo)

<Order total: 10.000000 due: 9.300000>

In [18]:
Order(joe, cart, large_order_promo)

<Order total: 42.000000 due: 42.000000>

## Choosing the Best Strategy: Simple Approach

---
### Example 6-5. Then _best_promo_ function applies all discounts and returns the largest

In [19]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)


In [20]:
Order(joe, long_order, best_promo)

<Order total: 10.000000 due: 9.300000>

In [21]:
Order(joe, banana_cart, best_promo)

<Order total: 30.000000 due: 28.500000>

In [22]:
Order(ann, cart, best_promo)

<Order total: 42.000000 due: 39.900000>

## Finding strategy in a Module

---
### Example 6-7. The promo list is built by introspection of the module global namespace

In [23]:
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']

def best_promo(order):
    return max([promo(order) for promo in promos])

[<function __main__.fidelity_promo(order: __main__.Order)>,
 <function __main__.bulk_item_promo(order: __main__.Order)>,
 <function __main__.large_order_promo(order: __main__.Order)>]

In [24]:
Order(joe, long_order, best_promo)

<Order total: 10.000000 due: 9.300000>

In [25]:
Order(joe, banana_cart, best_promo)

<Order total: 30.000000 due: 28.500000>

In [26]:
Order(ann, cart, best_promo)

<Order total: 42.000000 due: 39.900000>

In [27]:
gs = globals()
gs['fidelity_promo']

<function __main__.fidelity_promo(order: __main__.Order)>

---
### Example 6-8. The promo list is built by instrospection of a new _promotions_ module

In [28]:
import promotions
import inspect

# Inspect the promotions module searching functions
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    return max([promo(order) for promo in promos]) 

In [29]:
Order(joe, long_order, best_promo)

<Order total: 10.000000 due: 9.300000>

In [30]:
Order(joe, banana_cart, best_promo)

<Order total: 30.000000 due: 28.500000>

In [31]:
Order(ann, cart, best_promo)

<Order total: 42.000000 due: 39.900000>

## Command

---
### Example 6-9. Each instance of _MacroCommand_ has an internal list of commands

In [32]:
class MacroCommand:
    """A command that executes a list of commands"""

    def __init__(self, commands):
        self.commands = commands

    def __call__(self):
        for commmand in self.commands:
            command()

In [33]:
import registration

ModuleNotFoundError: No module named 'registration'